Evaluating the results from the survey and plotting them.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import random

import tqdm.notebook as tqdm
import numpy as np
import pandas as pd
import wandb
api = wandb.Api()

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from statannotations.Annotator import Annotator

In [ ]:
api = wandb.Api()
artifact = api.artifact(
    f"neural-audio-synthesis-thesis/neural-audio-synthesis-thesis/survey-backup:latest"
)

In [ ]:
BASE_DIR = "/Users/vaclav/prog/thesis/data/wandb-samples"
artifact.download(BASE_DIR)

In [ ]:
path = os.path.join(BASE_DIR, "mushra.csv")
df = pd.read_csv(path)

# two answers I used for testing
df = df.loc[df["session_uuid"] != "ef68e1d2-8b5f-4f14-abe4-70760956ade5"]
df = df.loc[df["session_uuid"] != "2299b6fa-6927-4564-ac71-e7887722fd16"]

# do not leak personal data
del df["email"]
del df["ethusername"]
del df["firstname"]
del df["lastname"]

from numpy.random import default_rng
rng = default_rng(seed=123)

groups = [df for _, df in df.groupby('session_uuid')]
random.shuffle(groups, random=rng.random)

df = pd.concat(groups).reset_index(drop=True)

# df["session_uuid"].unique()

In [ ]:
print("#respondents:", len(df["session_uuid"].unique()))

In [ ]:
df["rating_stimulus"] = df["rating_stimulus"].replace(
    {
        "0725-ddspae-2": "DDSP-full",
        "0725-ddspae-cnn-1-rt": "DDSP-CNN",
        "0809-ddspae-cnn-5-rt": "DDSP-CNN-Tiny",
        "0725-ddspae-cnn-1-rtq": "DDSP-CNN-Q",
        "0809-ddspae-cnn-5-rtq": "DDSP-CNN-Tiny-Q",
        "0805-ddspae": "DDSP-full",
        "0804-ddspae-cnn-3-rt": "DDSP-CNN",
        "0809-ddspae-cnn-4-rt": "DDSP-CNN-Tiny",
        "0804-ddspae-cnn-3-rtq": "DDSP-CNN-Q",
        "0809-ddspae-cnn-4-rtq": "DDSP-CNN-Tiny-Q",
        "reference": "Reference",
        "anchor35": "Anchor",
    }
)

In [ ]:
df["rating_stimulus"].unique()

In [ ]:
# An older version of the survey contains more questions, filter these out

responses_per_sample = df.loc[df["rating_stimulus"] == "Reference"].groupby("trial_id").count()["session_test_id"]
valid_samples = responses_per_sample[responses_per_sample > 6].index

df = df.loc[df["trial_id"].isin(responses_per_sample[responses_per_sample > 6].index)]

# Remove the participant training examples
df = df.loc[~df["trial_id"].isin(["trial1", "trial2"])]

In [ ]:
df2 = df.loc[
    (df["rating_stimulus"] == "Reference")
    & (~df["trial_id"].str.startswith("violin_tt"))
]
df3 = df.loc[
    (df["rating_stimulus"] == "Anchor")
    & (~df["trial_id"].str.startswith("violin_tt"))
]

In [ ]:
df2 = df2.copy()
df2["bad"] = df2["rating_score"] < 85  # 90 for a stricter version

df3 = df3.copy()
df3["bad"] = df3["rating_score"] > 90

In [ ]:
df2.groupby("session_uuid").agg({"bad": "mean"}).sort_values("bad", ascending=False)

In [ ]:
# A lot of people judge the anchor highly (maybe "naturalness" is not
# affected by a low-pass filter), let's not filter these out
df3.groupby("session_uuid").agg({"bad": "mean"})

In [ ]:
bad_part_per_participant = df2.groupby("session_uuid").agg({"bad": "mean"})
# Use 20% instead of 15% as in the recommendation because 1/6 ~= 0.16, barely above the limit
screened_participants = bad_part_per_participant.loc[bad_part_per_participant["bad"] >= 0.20].index
print(f"{len(screened_participants)} participants post-screened")

df = df.loc[~df["session_uuid"].isin(screened_participants)]

In [ ]:
all_trials = pd.Series(df["trial_id"].unique())
violin_trials = all_trials.loc[all_trials.str.startswith("violin_sample")]
trumpet_trials = all_trials.loc[all_trials.str.startswith("trumpet_sample")]
violin_tt_trials = all_trials.loc[all_trials.str.startswith("violin_tt_sample")]

In [ ]:
# Distribution of rating times, in seconds

rating_times = (
    df.loc[(df["rating_stimulus"] == "Reference")]
).copy()
rating_times["rating_time"] /= 1000

sns.boxplot(data=rating_times, y="trial_id", x="rating_time", showfliers=False)

In [ ]:
# Rating time (only for included samples) per participant, in minutes.
df.loc[(df["rating_stimulus"] == "Reference")].groupby("session_uuid").agg({"rating_time": "sum"}) / 1000 / 60

## Statistical evaluation

In [ ]:
from scipy.stats import wilcoxon


def make_agg_fn(condition_1, condition_2):
    """ Extract paired data for Wilcoxon signed rank test. """
    def agg_fn(df_cur):
        df_cur = df_cur.set_index("rating_stimulus")
        return pd.Series(
            [
                df_cur.loc[condition_1, "rating_score"],
                df_cur.loc[condition_2, "rating_score"],
            ]
        )

    return agg_fn

def get_p_value(condition_1, condition_2, trials):
    paired_ratings = df.loc[df["trial_id"].isin(trials)].groupby(["session_uuid", "trial_id"]).apply(
        make_agg_fn(condition_1, condition_2),
    )
    res = wilcoxon(
        paired_ratings.iloc[:,0],
        paired_ratings.iloc[:,1],
        correction=True,
#         alternative="less",
    )
    return res.pvalue

In [ ]:
def get_p_values(df, trials, full_matrix=True):
    """ Apply Wilcoxon to all pairs of conditions in a group of trials. """

    conditions = df.loc[df["trial_id"].isin(trials), "rating_stimulus"].unique()
    rows = []
    
    for condition1 in conditions:
        for condition2 in conditions:
            p = None
            if condition1 != condition2:
                p = get_p_value(condition1, condition2, trials)

            # if full_matrix is false, filter out duplicates and self-comparisons
            if full_matrix or condition1 < condition2:
                rows.append((condition1, condition2, p))

    p_values = pd.DataFrame(rows, columns=["condition_1", "condition_2", "p_value"])
    return p_values

Print the pairs with a significant p-value.

In [ ]:
p_threshold = 0.05

In [ ]:
p_values = get_p_values(df, violin_trials, full_matrix=False)

# Bonferroni correction
p_values["p_value"] *= len(p_values.loc[~p_values["p_value"].isna()])

p_values.loc[p_values["p_value"] < p_threshold]

In [ ]:
p_values = get_p_values(df, trumpet_trials, full_matrix=False)

# Bonferroni correction
p_values["p_value"] *= len(p_values.loc[~p_values["p_value"].isna()])

# p_values.loc[p_values["p_value"] < p_threshold * 10]
p_values

In [ ]:
p_values = get_p_values(df, violin_tt_trials, full_matrix=False)

# Bonferroni correction
corrected_p_threshold = 0.05 / len(p_values.loc[~p_values["p_value"].isna()])

p_values.loc[p_values["p_value"] < corrected_p_threshold]

## Boxplots

In [ ]:
plot_dir = "/Users/vaclav/prog/thesis/figures/"


def make_boxplot(
    df, trials, is_timbre_transfer=False, invert_annotations=False, pdf_name=None
):
    order = [
        "DDSP-full",
        "DDSP-CNN",
        "DDSP-CNN-Q",
        "DDSP-CNN-Tiny",
        "DDSP-CNN-Tiny-Q",
    ]

    if not is_timbre_transfer:
        order += ["Reference", "Anchor"]

    plt.figure(figsize=(6, 3))

    ax = sns.boxplot(
        data=df.loc[df["trial_id"].isin(trials)],
        y="rating_stimulus",
        x="rating_score",
        order=order,
    )

    #     plt.xticks(rotation=30)

    p_values = get_p_values(df, trials, full_matrix=False)
    # For timbre transfer, this removes the comparisons with (dummy) Reference
    p_values = p_values.loc[
        p_values["condition_1"].isin(order) & p_values["condition_2"].isin(order)
    ]

    # Bonferroni correction
    p_values["p_value"] *= len(p_values.loc[~p_values["p_value"].isna()])
    p_threshold = 0.05

    pairs = []
    p_values_list = []
    for _, row in p_values.iterrows():
        if row["p_value"] != np.nan:
            # If invert_annotations, show pairs *above* the threshold, i.e. insignificant ones
            if (row["p_value"] < p_threshold) != invert_annotations:
                pairs.append((row["condition_1"], row["condition_2"]))
                if invert_annotations:
                    p_values_list.append(0.1)
                else:
                    p_values_list.append(row["p_value"])

    annotator = Annotator(
        ax,
        pairs,
        data=df,
        x="rating_score",
        y="rating_stimulus",
        order=order,
        orient="h",
    )

    color = sns.color_palette()[3] if invert_annotations else (0.2, 0.2, 0.2)

    annotator.configure(
        test=None,
        test_short_name="Wilcoxon",
        verbose=1,
        color=color,
        show_test_name=True,
        #         pvalue_format={
        #             "pvalue_format_string": [
        #                 [1e-4, "****"],
        #                 [1e-3, "***"],
        #                 [1e-2, "**"],
        #                 [0.05, "*"],
        #                 [1, ""],  # wanted to remove ns but doesn't work
        #             ]
        #         },
    ).set_pvalues(pvalues=p_values_list).annotate()

    plt.ylabel("Condition")
    plt.xlabel("Score")
    plt.xticks(np.arange(0, 101, 20))

    test_label_x = (100 + plt.xlim()[1]) / 2
    test_label_y = plt.ylim()[0] + 0.25
    plt.text(
        test_label_x,
        test_label_y,
        (
            "Wilcoxon\n$p < 0.05$ pairs"
            if not invert_annotations
            else "Wilcoxon\n$p \geq 0.05$ pairs"
        ),
        ha="center",
        va="top",
        fontstyle="italic",
    )

    if pdf_name is not None:
        pdf_metadata = {
            "artifact-version": artifact.version,
            "artifact-created-at": artifact.created_at,
            "respondents-after-screening": len(df["session_uuid"].unique()),
        }

        plt.savefig(
            os.path.join(plot_dir, pdf_name),
            bbox_inches="tight",
            # Metadata doc: https://matplotlib.org/stable/api/backend_pdf_api.html#matplotlib.backends.backend_pdf.PdfFile
            metadata={"Title": " ".join([f"{k}={v}" for k, v in pdf_metadata.items()])},
            backend="pgf",
        )


#     annotator.configure(test='Wilcoxon', text_format='star', loc='outside')
#     annotator.apply_and_annotate()

In [ ]:
# make_boxplot(df, violin_trials, invert_annotations=True)
make_boxplot(df, violin_trials, invert_annotations=True, pdf_name="survey_violin.pdf")

In [ ]:
make_boxplot(df, trumpet_trials, pdf_name="survey_trumpet.pdf")

In [ ]:
make_boxplot(df, violin_tt_trials, is_timbre_transfer=True, pdf_name="survey_violin_tt.pdf")

## Demographics

In [ ]:
audio_familiarity = df.groupby("session_uuid").agg(
    {"musical-instruments": "first", "other-audio": "first"}
)
sns.displot(
    data=audio_familiarity.sort_values("musical-instruments"), x="musical-instruments"
)

sns.displot(data=audio_familiarity.sort_values("other-audio"), x="other-audio")